# Welcome to the API!

**Goal**: Easy programmatic access to the database with **no user SQL**


## Notes

 * This is not a REST API, more of an SDK
 * Current access is for *point* and *layer* data
 * Funtions return **lists** or **Geopandas Dataframes**

### Step 1. Import the classes, explore them

In [ ]:
# imports
from datetime import date
import geopandas as gpd
from snowexsql.api import PointMeasurements, LayerMeasurements

In [ ]:
# The main functions we will use are `from_area` and `from_filter` like this
df = PointMeasurements.from_filter(
    date=date(2020, 5, 28), instrument='camera'
)
df.head()

#### Notice:
 * We did not need to manage SQL
 * We got a geopandas array
 * We filtered on specific attributes known to be in the database

#### How do I know what to filter by?

In [ ]:
# Find what you can filter by
print(PointMeasurements.ALLOWED_QRY_KWARGS)
print(LayerMeasurements.ALLOWED_QRY_KWARGS)

#### How do I know what values work for filtering?

In [ ]:
print(PointMeasurements().all_observers)

### Try it out

* What instrument could you filter by for PointData?
* What site names could you filter by for LayerData?

Notice we instantiate the class 
`PointMeasurements()`
Before calling the property `.all_observers`

In [ ]:
# <YOUR CODE HERE>

In [ ]:
# Explore the points
df.crs
df.to_crs("EPSG:4326").loc[:,["id", "value", "type", "geom", "instrument"]].explore()

#### What if I have a point or a shapefile

Both the PointMeasurement and LayerMeasurement class have a function called `from_area`
that takes either a `shapely` polygon or a `shapely` point and a radius as well as the same
filter kwargs available in `.from_filter`


In [ ]:
# Set up a fake shapefile
gdf = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy(
        [743766.4794971556], [4321444.154620216], crs="epsg:26912"
    ).buffer(2000.0)
).set_crs("epsg:26912")

# This is the area we will filter to
gdf.explore()

In [ ]:
# Get density near the point
df = LayerMeasurements.from_area(
    type="density",
    shp=gdf.iloc[0].geometry,
)

df.to_crs("EPSG:4326").loc[:,["id", "depth", "value", "type", "geom"]].explore()

### How much filtering is enough? 

I got a `LargeQueryCheckException`

GIVE ME THE DATA PLEASE

In [ ]:
# This query will fail
df = PointMeasurements.from_filter(
    instrument="magnaprobe",
)

In [ ]:
# Th queries will pass
df = PointMeasurements.from_filter(
    instrument="magnaprobe",
    limit=100
)

df.head()

### DANGER ZONE
If you need more than 1000 points returned, you can specify so with the `limit`

The intention is to be aware of how much data will be returned

In [ ]:
# DANGER ZONE
# If you need more than 1000 points returned, you can specify so with the limit
df = PointMeasurements.from_filter(
    date=date(2020, 1, 28),
    instrument="magnaprobe",
    limit=3000
)
df.head()

# THE END

### Go forth and explore